# Semi-Supervised Learning — Revenue Tier Classification

**Goal:** Compare a supervised baseline against three semi-supervised methodologies and select the best-performing model for revenue tier prediction, evaluated on macro F1 on a held-out labeled test set.

**Dataset:** `data_ssl_revenue.csv` produced by the Feature Engineering pipeline.

**Target:** `y_ssl`
- 0 = Low, 1 = Medium, 2 = High, 3 = Blockbuster (labeled)
- -1 = unlabeled

**Models:**
1. GradientBoosting (supervised baseline)
2. RandomForest (supervised baseline)
3. SelfTraining (semi-supervised)
4. LabelSpreading (semi-supervised, graph-based)
5. LabelPropagation (semi-supervised, graph-based)

**Theoretical guarantees:**
- Train/test split on labeled data only
- Test set never participates in pseudo-labeling
- Unlabeled data used only during SSL fitting
- No post-release variables as features
- Scaler fitted only on labeled train
- PCA (if used) fitted only on training data
- All models evaluated on the same labeled test set
- Primary metric: macro F1

In [ ]:
# ============================================================
# Imports
# ============================================================
import pandas as pd
import numpy as np
import warnings
import os
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.semi_supervised import (
    SelfTrainingClassifier,
    LabelSpreading,
    LabelPropagation,
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

warnings.filterwarnings("ignore")
print("All imports successful.")

## 0. Load Dataset

In [ ]:
# ============================================================
# Load the SSL revenue dataset
# ============================================================
DATA_DIR = "../data"
ARTIFACTS_DIR = "../data"  # save artifacts alongside datasets

df = pd.read_csv(os.path.join(DATA_DIR, "data_ssl_revenue.csv"))
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\ny_ssl value counts:\n{df['y_ssl'].value_counts().sort_index()}")

---
## Validation 1 — Leakage Prevention

Remove any column whose name contains post-release or target keywords (`vote`, `review`, `rating`, `popularity`, `revenue`, `budget`). Keep `y_ssl` only as the target.

In [ ]:
# ============================================================
# Validation 1 — Leakage prevention
# ============================================================
leakage_keywords = ["vote", "review", "rating", "popularity", "revenue", "budget"]

# All columns except y_ssl are candidate features
candidate_features = [c for c in df.columns if c != "y_ssl"]

# Remove any column whose name contains a leakage keyword
leaked_cols = [c for c in candidate_features
               if any(kw in c.lower() for kw in leakage_keywords)]

feature_cols = [c for c in candidate_features if c not in leaked_cols]

print(f"Candidate features before filter: {len(candidate_features)}")
print(f"Leaked columns removed ({len(leaked_cols)}): {leaked_cols}")
print(f"\nFinal feature count: {len(feature_cols)}")
print(f"First 20 features: {feature_cols[:20]}")

## Validation 2 — Class Distribution

In [ ]:
# ============================================================
# Validation 2 — Class distribution (labeled data)
# ============================================================
labeled_mask = df["y_ssl"] != -1
df_labeled = df[labeled_mask].copy()

class_counts = df_labeled["y_ssl"].value_counts().sort_index()
print(f"Class distribution (labeled rows):")
tier_names = {0: "Low", 1: "Medium", 2: "High", 3: "Blockbuster"}
for cls, cnt in class_counts.items():
    print(f"  {cls} ({tier_names.get(cls, '?')}): {cnt}")

n_classes = df_labeled["y_ssl"].nunique()
print(f"\nNumber of classes: {n_classes}")
assert n_classes == 4, f"Expected 4 classes, got {n_classes}"
print(f"Total labeled rows: {len(df_labeled)}")

## Validation 3 — Unlabeled Proportion

In [ ]:
# ============================================================
# Validation 3 — Unlabeled proportion
# ============================================================
unlabeled_mask = df["y_ssl"] == -1
n_unlabeled = unlabeled_mask.sum()
pct_unlabeled = n_unlabeled / len(df) * 100

print(f"Unlabeled rows:  {n_unlabeled}")
print(f"Percentage:      {pct_unlabeled:.2f}%")
print(f"Labeled rows:    {(~unlabeled_mask).sum()}")
print(f"Total rows:      {len(df)}")

## Validation 4 — Feature Dimensionality

In [ ]:
# ============================================================
# Validation 4 — Feature dimensionality
# ============================================================
n_features = len(feature_cols)
USE_PCA = n_features > 100

print(f"Total feature columns: {n_features}")
if USE_PCA:
    print(">> Feature count > 100 — will apply PCA (95% variance) for graph-based SSL methods.")
else:
    print(">> Feature count <= 100 — no PCA needed for graph-based methods.")

## Validation 5 — Missing Features

In [ ]:
# ============================================================
# Validation 5 — Missing features
# ============================================================
X_all = df[feature_cols].copy()
y_all = df["y_ssl"].copy()

missing_before = X_all.isnull().sum()
total_missing_before = missing_before.sum()
print(f"Total missing values BEFORE imputation: {total_missing_before}")

if total_missing_before > 0:
    print(f"\nMissing per column (non-zero):")
    print(missing_before[missing_before > 0])
    
    # Apply median imputation for numeric columns
    imputer = SimpleImputer(strategy="median")
    X_all_imputed = pd.DataFrame(
        imputer.fit_transform(X_all),
        columns=feature_cols,
        index=X_all.index
    )
    total_missing_after = X_all_imputed.isnull().sum().sum()
    print(f"\nTotal missing values AFTER imputation: {total_missing_after}")
    X_all = X_all_imputed
else:
    print("No missing values — no imputation needed.")

print(f"\nFinal feature matrix shape: {X_all.shape}")

---
# Experiment

## Step 1 — Split Labeled Data

In [ ]:
# ============================================================
# Step 1 — Split labeled data only
# ============================================================

# Separate labeled and unlabeled feature matrices
X_labeled = X_all[labeled_mask].copy()
y_labeled = y_all[labeled_mask].copy()
X_unlabeled = X_all[unlabeled_mask].copy()

print(f"Labeled:   X={X_labeled.shape}  y={y_labeled.shape}")
print(f"Unlabeled: X={X_unlabeled.shape}")

# Stratified train/test split on labeled data
X_train_labeled, X_test, y_train, y_test = train_test_split(
    X_labeled, y_labeled,
    test_size=0.2,
    random_state=42,
    stratify=y_labeled
)

print(f"\nAfter split:")
print(f"  X_train_labeled: {X_train_labeled.shape}")
print(f"  X_test:          {X_test.shape}")
print(f"  y_train:         {y_train.shape}  distribution: {dict(y_train.value_counts().sort_index())}")
print(f"  y_test:          {y_test.shape}  distribution: {dict(y_test.value_counts().sort_index())}")
print(f"  X_unlabeled:     {X_unlabeled.shape}")
print(f"\n>> Test set ({len(y_test)} rows) will NEVER be used during SSL fitting.")

## Step 2 — Preprocessing (StandardScaler)

In [ ]:
# ============================================================
# Step 2 — Fit StandardScaler on labeled train ONLY
# ============================================================
scaler = StandardScaler()
scaler.fit(X_train_labeled)

X_train_s = pd.DataFrame(
    scaler.transform(X_train_labeled),
    columns=feature_cols, index=X_train_labeled.index
)
X_test_s = pd.DataFrame(
    scaler.transform(X_test),
    columns=feature_cols, index=X_test.index
)
X_unlabeled_s = pd.DataFrame(
    scaler.transform(X_unlabeled),
    columns=feature_cols, index=X_unlabeled.index
)

print("StandardScaler fitted on labeled training data only.")
print(f"  X_train_s:     {X_train_s.shape}")
print(f"  X_test_s:      {X_test_s.shape}")
print(f"  X_unlabeled_s: {X_unlabeled_s.shape}")

## Step 3 — Supervised Baselines

In [ ]:
# ============================================================
# Helper: evaluate a model and return metrics dict
# ============================================================
def evaluate_model(model, X_eval, y_eval, model_name, notes=""):
    """
    Predict on X_eval and compute accuracy, macro_f1, confusion matrix.
    Returns a dict suitable for the comparison table.
    """
    y_pred = model.predict(X_eval)
    acc = accuracy_score(y_eval, y_pred)
    f1 = f1_score(y_eval, y_pred, average="macro")
    cm = confusion_matrix(y_eval, y_pred)
    
    print(f"\n{'='*60}")
    print(f"  {model_name}")
    print(f"{'='*60}")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Macro F1:  {f1:.4f}")
    print(f"\n  Classification Report:")
    print(classification_report(y_eval, y_pred, target_names=["Low", "Medium", "High", "Blockbuster"]))
    print(f"  Confusion Matrix:")
    print(cm)
    
    return {
        "model_name": model_name,
        "accuracy": round(acc, 4),
        "macro_f1": round(f1, 4),
        "notes": notes,
        "_model": model,
        "_y_pred": y_pred,
        "_cm": cm,
    }

print("Evaluation helper defined.")

In [ ]:
# ============================================================
# Step 3a — Gradient Boosting (supervised baseline)
# ============================================================
results = []

gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_train_s, y_train)

res_gb = evaluate_model(gb_clf, X_test_s, y_test,
                        "GradientBoosting (supervised)",
                        notes="Supervised baseline")
results.append(res_gb)

In [ ]:
# ============================================================
# Step 3b — Random Forest (supervised baseline)
# ============================================================
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_s, y_train)

res_rf = evaluate_model(rf_clf, X_test_s, y_test,
                        "RandomForest (supervised)",
                        notes="Supervised baseline")
results.append(res_rf)

## Step 4 — Prepare Semi-Supervised Training Data

In [ ]:
# ============================================================
# Step 4 — Concatenate labeled train + unlabeled for SSL
# ============================================================
X_train_ssl = pd.concat([X_train_s, X_unlabeled_s], axis=0).reset_index(drop=True)
y_train_ssl = pd.concat([
    y_train.reset_index(drop=True),
    pd.Series([-1] * len(X_unlabeled_s))
], axis=0).reset_index(drop=True)

print(f"SSL training data:")
print(f"  X_train_ssl: {X_train_ssl.shape}")
print(f"  y_train_ssl: {y_train_ssl.shape}")
print(f"  Labeled in SSL train:   {(y_train_ssl != -1).sum()}")
print(f"  Unlabeled in SSL train: {(y_train_ssl == -1).sum()}")
print(f"\n>> Test set ({len(y_test)} rows) is EXCLUDED from SSL training data.")

## Step 5 — Semi-Supervised Models

### Model A: Self-Training

In [ ]:
# ============================================================
# Model A — Self-Training with RandomForest base estimator
# ============================================================
st_base = RandomForestClassifier(random_state=42)
st_clf = SelfTrainingClassifier(
    base_estimator=st_base,
    threshold=0.9,
    verbose=False
)
st_clf.fit(X_train_ssl, y_train_ssl)

# Count pseudo-labeled samples
if hasattr(st_clf, "labeled_iter_"):
    # labeled_iter_ > 0 means pseudo-labeled in that iteration
    n_pseudo = int((st_clf.labeled_iter_ > 0).sum())
    st_notes = f"Pseudo-labeled: {n_pseudo} samples"
else:
    n_pseudo = "N/A"
    st_notes = "Pseudo-label count not available"

print(f"Self-Training complete. {st_notes}")

res_st = evaluate_model(st_clf, X_test_s, y_test,
                        "SelfTraining (SSL)",
                        notes=st_notes)
results.append(res_st)

### PCA Preparation for Graph-Based Methods

LabelSpreading and LabelPropagation use distance-based kernels. If the feature dimensionality is high (>100), we reduce it with PCA (retaining 95% explained variance). PCA is fitted **only on the SSL training data** to prevent leakage.

In [ ]:
# ============================================================
# PCA preparation for graph-based SSL methods (if needed)
# ============================================================
if USE_PCA:
    print(f"Feature count = {n_features} > 100 — applying PCA (95% variance)")
    pca = PCA(n_components=0.95, random_state=42)
    pca.fit(X_train_ssl)  # fit only on SSL training data
    
    X_train_ssl_pca = pca.transform(X_train_ssl)
    X_test_s_pca = pca.transform(X_test_s)
    
    print(f"  PCA components retained: {pca.n_components_}")
    print(f"  Explained variance:      {pca.explained_variance_ratio_.sum():.4f}")
    print(f"  X_train_ssl_pca shape:   {X_train_ssl_pca.shape}")
    print(f"  X_test_s_pca shape:      {X_test_s_pca.shape}")
else:
    print(f"Feature count = {n_features} <= 100 — using original features for graph-based methods.")
    X_train_ssl_pca = X_train_ssl.values
    X_test_s_pca = X_test_s.values
    pca = None

### Model B: Label Spreading

In [ ]:
# ============================================================
# Model B — Label Spreading (graph-based, KNN kernel)
# ============================================================
ls_clf = LabelSpreading(kernel="knn", n_neighbors=10)
ls_clf.fit(X_train_ssl_pca, y_train_ssl)

pca_note = f"PCA to {X_train_ssl_pca.shape[1]}d" if USE_PCA else "No PCA"

res_ls = evaluate_model(ls_clf, X_test_s_pca, y_test,
                        "LabelSpreading (SSL)",
                        notes=f"kernel=knn, n_neighbors=10, {pca_note}")
results.append(res_ls)

### Model C: Label Propagation

In [ ]:
# ============================================================
# Model C — Label Propagation (graph-based, KNN kernel)
# ============================================================
lp_clf = LabelPropagation(kernel="knn", n_neighbors=10)
lp_clf.fit(X_train_ssl_pca, y_train_ssl)

res_lp = evaluate_model(lp_clf, X_test_s_pca, y_test,
                        "LabelPropagation (SSL)",
                        notes=f"kernel=knn, n_neighbors=10, {pca_note}")
results.append(res_lp)

---
## Step 6 — Comparison & Best Model Selection

In [ ]:
# ============================================================
# Step 6 — Comparison table & best model selection
# ============================================================
comparison_df = pd.DataFrame([
    {"model_name": r["model_name"], "accuracy": r["accuracy"],
     "macro_f1": r["macro_f1"], "notes": r["notes"]}
    for r in results
])

# Sort by macro_f1 (primary) then accuracy (tiebreaker), descending
comparison_df = comparison_df.sort_values(
    ["macro_f1", "accuracy"], ascending=[False, False]
).reset_index(drop=True)

print("="*70)
print("  MODEL COMPARISON TABLE")
print("="*70)
print(comparison_df.to_string(index=False))

# Select best model
best_idx = comparison_df.index[0]
best_name = comparison_df.loc[best_idx, "model_name"]
best_f1 = comparison_df.loc[best_idx, "macro_f1"]
best_acc = comparison_df.loc[best_idx, "accuracy"]
best_result = [r for r in results if r["model_name"] == best_name][0]
best_model = best_result["_model"]

print(f"\n>> BEST MODEL: {best_name}")
print(f"   Macro F1:  {best_f1}")
print(f"   Accuracy:  {best_acc}")

## Step 7 — Save Artifacts

In [ ]:
# ============================================================
# Step 7a — Save comparison table
# ============================================================
comp_path = os.path.join(ARTIFACTS_DIR, "ssl_model_comparison.csv")
comparison_df.to_csv(comp_path, index=False)
print(f"Saved comparison table: {comp_path}")

In [ ]:
# ============================================================
# Step 7b — Save best model and scaler
# ============================================================
model_path = os.path.join(ARTIFACTS_DIR, "best_ssl_model.joblib")
scaler_path = os.path.join(ARTIFACTS_DIR, "ssl_scaler.joblib")

joblib.dump(best_model, model_path)
print(f"Saved best model ({best_name}): {model_path}")

joblib.dump(scaler, scaler_path)
print(f"Saved scaler: {scaler_path}")

# Also save PCA if it was used
if pca is not None:
    pca_path = os.path.join(ARTIFACTS_DIR, "ssl_pca.joblib")
    joblib.dump(pca, pca_path)
    print(f"Saved PCA: {pca_path}")

In [ ]:
# ============================================================
# Step 7c — Confusion matrix plot for best model
# ============================================================
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay(
    confusion_matrix=best_result["_cm"],
    display_labels=["Low", "Medium", "High", "Blockbuster"]
).plot(ax=ax, cmap="Blues", values_format="d")
ax.set_title(f"Confusion Matrix — {best_name}\nMacro F1 = {best_f1}  |  Accuracy = {best_acc}", fontsize=13)
plt.tight_layout()

cm_path = os.path.join(ARTIFACTS_DIR, "best_ssl_confusion_matrix.png")
fig.savefig(cm_path, dpi=150, bbox_inches="tight")
print(f"Saved confusion matrix plot: {cm_path}")
plt.show()

---
## Summary

| Step | Description |
|---|---|
| Validation 1 | Leakage prevention — removed post-release features |
| Validation 2 | Confirmed 4 revenue-tier classes |
| Validation 3 | Documented unlabeled proportion |
| Validation 4 | Checked dimensionality for PCA decision |
| Validation 5 | Imputed missing values if any |
| Split | 80/20 stratified on labeled data only |
| Scaling | StandardScaler fitted on labeled train |
| Baselines | GradientBoosting & RandomForest (supervised) |
| SSL Models | SelfTraining, LabelSpreading, LabelPropagation |
| Selection | Best model by macro F1 (accuracy as tiebreaker) |
| Artifacts | Comparison CSV, best model, scaler, confusion matrix |

**All evaluation was performed on a held-out labeled test set that never participated in pseudo-labeling.**